# Aula 09.Ex01 - Malhas e Texturas - Mapeamento de Texturas

### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image

In [2]:
from lib import Object3D
from lib import Transform3D

### Inicializando janela

In [3]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
altura = 1080
largura = 1920
window = glfw.create_window(largura, altura, "T2 - CG", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec3.
* Definindo matrizes Model, View e Projection que acumulam transformações geométricas 3D e permitem navegação no cenário.
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base nas transformações Model, View e Projection.

In [4]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        varying vec2 out_texture;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [5]:
fragment_code = """
        uniform vec4 color;
        varying vec2 out_texture;
        uniform sampler2D samplerTexture;
        
        void main(){
            vec4 texture = texture2D(samplerTexture, out_texture);
            gl_FragColor = texture;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [6]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [7]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [8]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [9]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [10]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [11]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Carregando Modelos (vértices e texturas) a partir de Arquivos

A função abaixo carrega modelos a partir de arquivos no formato WaveFront.


Para saber mais sobre o modelo, acesse: https://en.wikipedia.org/wiki/Wavefront_.obj_file


Nos slides e vídeo-aula da Aula 11 - Parte 1, nós descrevemos o funcionamento desse formato.

In [12]:
glEnable(GL_TEXTURE_2D)
qtd_texturas = 15
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

### Vamos carregar cada modelo e definir funções para desenhá-los

In [13]:
### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(0, 'models/stone_floor/txt_stone_floor.jpg')

stone_floor = Object3D.Object3D()
stone_floor.t_y = -1.01
stone_floor.s_x = 10.0
stone_floor.s_y = 10.0
stone_floor.s_z = 10.0
stone_floor.load_from_file_with_texture('models/stone_floor/mdl_stone_floor.obj', [0])

In [14]:
### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(3, 'models/house/txt_house_1.jpg')
load_texture_from_file(2, 'models/house/txt_house_2.png')
load_texture_from_file(1, 'models/house/txt_house_3.png')

house = Object3D.Object3D()
house.t_y = -1.0
house.s_x = 0.02
house.s_y = 0.02
house.s_z = 0.02
house.load_from_file_with_texture('models/house/mdl_house.obj', [1,2,3])

In [15]:
### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(4, 'models/table/txt_table.png')

table = Object3D.Object3D()
table.t_y = -1.0
table.load_from_file_with_texture('models/table/mdl_table.obj', [4,4])

In [16]:
### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(5, 'models/spiderman/txt_spiderman.png')

spiderman = Object3D.Object3D()
spiderman.t_y = -1.0
spiderman.t_z = -1.1
spiderman.t_x = -1.3
spiderman.a_y = 55
spiderman.s_x = 0.15
spiderman.s_y = 0.15
spiderman.s_z = 0.15
spiderman.load_from_file_with_texture('models/spiderman/mdl_spiderman.obj', [5])

In [17]:
### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(6, 'models/tree/txt_tree_1.jpg')
load_texture_from_file(7, 'models/tree/txt_tree_2.png')

tree = Object3D.Object3D()
tree.t_y = -1.0
tree.t_z = -3.1
tree.t_x = +7.0
tree.s_x = 0.75
tree.s_y = 0.75
tree.s_z = 0.75
tree.load_from_file_with_texture('models/tree/mdl_tree.obj', [6,7])

In [18]:
### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(8, 'models/tanks/txt_tanks.jpg')

tanks = Object3D.Object3D()
tanks.t_y = -1.0
tanks.t_z = -7
tanks.t_x = -7.0
tanks.a_x = -90
tanks.s_x = 0.5
tanks.s_y = 0.5
tanks.s_z = 0.5
tanks.load_from_file_with_texture('models/tanks/mdl_tanks.obj', [8,8,8,8,8])

In [19]:
### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(9, 'models/plane/txt_plane_1.jpg')
load_texture_from_file(10, 'models/plane/txt_plane_2.jpg')

plane = Object3D.Object3D()
plane.t_y = 10.0
plane.t_x = -20.0
plane.a_x = -90
plane.a_y = +30
plane.s_x = 0.001
plane.s_y = 0.001
plane.s_z = 0.001
plane.load_from_file_with_texture('models/plane/mdl_plane.obj', [9,10])

In [20]:
### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(11, 'models/sky/txt_sky.png')

sky_box = Object3D.Object3D()
sky_box.s_x = 2.0
sky_box.s_y = 2.0
sky_box.s_z = 2.0
sky_box.load_from_file_with_texture('models/sky/mdl_sky.obj', [11])

In [21]:
### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(13, 'models/notebook/txt_notebook_1.jpg')
load_texture_from_file(12, 'models/notebook/txt_notebook_2.jpg')

notebook = Object3D.Object3D()
notebook.t_y = -0.64
notebook.s_x = 0.1
notebook.s_y = 0.1
notebook.s_z = 0.1
notebook.load_from_file_with_texture('models/notebook/mdl_notebook.obj', [12, 13])

In [22]:
### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(14, 'models/grass/txt_grass.jpg')

grass = Object3D.Object3D()
grass.t_y = -1.011
grass.s_x = 100.0
grass.s_y = 100.0
grass.s_z = 100.0
grass.load_from_file_with_texture('models/grass/mdl_grass.obj', [14])

In [23]:
all_objects = [stone_floor, grass, sky_box, house, table, spiderman, tree, tanks, plane, notebook]

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar slots.

Nós agora vamos requisitar dois slots.
* Um para enviar coordenadas dos vértices.
* Outros para enviar coordenadas de texturas.

In [24]:
# Request a buffer slot from GPU
buffer = glGenBuffers(2)

serialized_vertices, serialized_tex_coords = Object3D.serialize_objects3d(all_objects)

###  Enviando coordenadas de vértices para a GPU

In [25]:
vertices = np.zeros(len(serialized_vertices), [("position", np.float32, 3)])
vertices['position'] = serialized_vertices


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)


###  Enviando coordenadas de textura para a GPU

In [26]:
textures = np.zeros(len(serialized_tex_coords), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = serialized_tex_coords


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)


### Eventos para modificar a posição da câmera.

* Usei as teclas A, S, D e W para movimentação no espaço tridimensional
* Usei a posição do mouse para "direcionar" a câmera

In [27]:
cameraPos   = glm.vec3(0.0,  0.0,  1.0);
cameraFront = glm.vec3(0.0,  0.0, -1.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);

skybox_radius = 50

polygonal_mode = False

def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode
    
    cameraSpeed = 0.2
    if key == 87 and (action==1 or action==2): # tecla W
        newCamPos = cameraPos + cameraSpeed * cameraFront
        if(glm.length(newCamPos) < skybox_radius):
            cameraPos = newCamPos
    
    if key == 83 and (action==1 or action==2): # tecla S
        newCamPos = cameraPos - cameraSpeed * cameraFront
        if(glm.length(newCamPos) < skybox_radius):
            cameraPos = newCamPos
    
    if key == 65 and (action==1 or action==2): # tecla A
        newCamPos = cameraPos - glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        if(glm.length(newCamPos) < skybox_radius):
            cameraPos = newCamPos
        
    if key == 68 and (action==1 or action==2): # tecla D
        newCamPos = cameraPos + glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        if(glm.length(newCamPos) < skybox_radius):
            cameraPos = newCamPos
        
    if key == 80 and action==1 and polygonal_mode==True:
        polygonal_mode=False
    else:
        if key == 80 and action==1 and polygonal_mode==False:
            polygonal_mode=True
            
        
        
        
firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.1 
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;

    
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)


glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED);
glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)




### Matrizes Model, View e Projection

Teremos uma aula específica para entender o seu funcionamento.

In [28]:
def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

### Nesse momento, nós exibimos a janela!


### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

In [ ]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

glEnable(GL_DEPTH_TEST) ### importante para 3D

# plane.r_y = 1.0
# plane.r_x = 0.0
plane_angle = 0
   

while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    if polygonal_mode==True:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    if polygonal_mode==False:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)
        
    plane_angle -= 0.1
    plane_extra_mat = Transform3D.rotacao(plane_angle, (0, 0, 0), 'y')
    
    loc_model = glGetUniformLocation(program, "model")
    
    offset = 0
    for obj in all_objects:
        if(obj == plane):
            offset = obj.render(loc_model, offset, [plane_extra_mat])
        else:
            offset = obj.render(loc_model, offset)

    
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_FALSE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_FALSE, mat_projection)    
    
    

    
    glfw.swap_buffers(window)

glfw.terminate()